In [1]:
import matplotlib.pyplot as plt
import torch
from botorch.models import ModelListGP
from botorch.optim import optimize_acqf
from torch.quasirandom import SobolEngine
from matplotlib import pyplot as plt
import numpy as np
from alse.eci import ExpectedCoverageImprovement
from alse.accuracy import get_accuracy
from alse.utils import identify_samples_which_satisfy_constraints, store_param, transform, un_transform
from alse.fit_model import fit_gp_class, fit_gp_reg
from alse.test_function.fxns import yf, hills, circle
from alse.fit_all_demo_only import fit_all_models
# from mpl_toolkits import mplot3d
# from matplotlib import cm

In [2]:
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}
            #   width > 2.5   powder capture > 50%     place holder
constraints = [("gt", 2.5),      ("gt", 0.5),         ("gt", 0.1)]

Read and consolidate data

In [3]:
import pandas
data1 = pandas.read_excel('4340_2.5mm_5.24gmin.xlsx').loc
data2 = pandas.read_excel('4340_3mm_5.24gmin.xlsx').loc
data3 = pandas.read_excel('4340_3mm_10.47gmin.xlsx').loc
consolidated_data = [data1, data2, data3]

In [4]:
ones = torch.ones(40)

In [5]:
def read_data(raw_data, param):
    d = torch.tensor([])
    for i in range(len(raw_data)):
        d = torch.cat((d, torch.tensor(raw_data[i][:,f"{param}"])))
    return d

In [6]:
power = read_data(consolidated_data, "P (W)")
velocity = (read_data(consolidated_data, "V (mm/min)"))

#This needs to change if using different dataset
spot_size = torch.cat((ones*2.5, ones*3, ones*3))
feed_rate = torch.cat((ones*5.24, ones*5.24, ones*10.47))

X = torch.stack((power, velocity, spot_size, feed_rate), -1)

In [7]:
X.shape

torch.Size([120, 4])

In [8]:
width = read_data(consolidated_data, "widths avg (mm)").unsqueeze(-1)
pow_cap = (read_data(consolidated_data, "powder capt %")/100).unsqueeze(-1)
_height = read_data(consolidated_data, "heights avg (mm)").bool().long().unsqueeze(-1)
adhere = torch.cat(([_height]*4),-1)

In [9]:
# Store for un_transform later
xrange_list, xmin_list = store_param(X, 4)

In [10]:
xrange_list

tensor([2.4000e+03, 2.8000e+03, 5.0000e-01, 5.2300e+00])

In [11]:
X = transform(X, 4)

In [12]:
X = un_transform(X, xrange_list, xmin_list, 4)

In [13]:
gp_model_width = fit_gp_reg(X, width)
gp_model_pow_cap = fit_gp_reg(X, pow_cap)
gp_model_adhere = fit_gp_class(X, adhere, lr=0.05, iter=300)


C:\Users\Leo\AppData\Roaming\Python\Python310\site-packages\botorch\fit.py:148: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(w.message, w.category)
c:\Program Files\Python310\lib\site-packages\gpytorch\lazy\triangular_lazy_tensor.py:136: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


In [14]:
# N1, N2 = 150, 150
# Xplt, Yplt, xplt = make_meshgrid(N1, N2, [0, 1], [0, 1], tkwargs)

In [15]:
gp_model_width.covar_module.base_kernel.lengthscale

tensor([[0.6928, 0.6930, 0.6972, 0.6930]], grad_fn=<SoftplusBackward0>)

In [16]:
punchout_radius = 0.1
bounds = torch.tensor([[200, 300, 0.5, 1], [3000, 3000, 5, 12]], **tkwargs)
lb, ub = bounds


In [17]:

# We don't have to normalize X since the domain is [0, 1]^2. Make sure to
# appropriately adjust the punchout radius if the domain is normalized.

model_list_gp = ModelListGP(gp_model_width, gp_model_pow_cap, gp_model_adhere)

eci = ExpectedCoverageImprovement(
    model=model_list_gp,
    constraints=constraints,
    punchout_radius=punchout_radius,
    bounds=bounds,
    num_samples=512
)

# gp_model_width.eval()
# gp_model_pow_cap.eval()
# gp_model_adhere.eval()
for model in model_list_gp.models:
    model.eval()

x_next, _ = optimize_acqf(
    acq_function=eci,
    bounds=bounds,
    q=1,
    num_restarts=10,
    raw_samples=512
)
new_power = round(x_next[0][0].item(), 2)
new_velocity = round(x_next[0][1].item(), 2)
new_spotsize = round(x_next[0][2].item(), 2)
new_feedrate = round(x_next[0][3].item(), 2)
print(f"New X: Power: {new_power}W, Velocity: {new_velocity}mm/min, Spot size: {new_spotsize}mm, Feedrate: {new_feedrate}g/min")

c:\Program Files\Python310\lib\site-packages\gpytorch\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\gpytorch\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\gpytorch\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(


New X: Power: 2113.01W, Velocity: 2198.54mm/min, Spot size: 2.01mm, Feedrate: 8.94g/min
